<a href="https://colab.research.google.com/github/cappyx/MISSP/blob/main/challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Helper function to print the solution
def print_solution(p):
    p.solve()
    print("Status:", LpStatus[p.status])
    for v in p.variables():
        print(v.name, "=", v.varValue)
    print("Objective value =", value(p.objective))

# Challenge 1

In [4]:
import pulp
from pulp import *

# Resistances
r1, r2, r3, r4, r5 = 8, 6, 4, 10, 8

# Maximum currents through individual resistors
i1_max, i2_max, i3_max, i4_max, i5_max = 2, 3, 4, 2, 2

# Create the optimization problem (maximization)
prob_current = LpProblem("Maximum_Circuit_Current", LpMaximize)

# Decision variables:
# I_total - total current flowing through the circuit (input current to the series connection)
# I_r1, I_r2, I_r3, I_r4, I_r5 - currents flowing through individual resistors
I_total = LpVariable("Total_Current", lowBound=0)
I_r1 = LpVariable("Current_R1", lowBound=0)
I_r2 = LpVariable("Current_R2", lowBound=0)
I_r3 = LpVariable("Current_R3", lowBound=0)
I_r4 = LpVariable("Current_R4", lowBound=0)
I_r5 = LpVariable("Current_R5", lowBound=0)

# Objective function: maximize total current
prob_current += I_total, "Maximize_Total_Current"

# Constraints:

# 1. Maximum current constraints for individual resistors
prob_current += I_r1 <= i1_max, "Max_Current_R1"
prob_current += I_r2 <= i2_max, "Max_Current_R2"
prob_current += I_r3 <= i3_max, "Max_Current_R3"
prob_current += I_r4 <= i4_max, "Max_Current_R4"
prob_current += I_r5 <= i5_max, "Max_Current_R5"

# 2. Kirchhoff's Current Laws (KCL) and Ohm's Law

# Total current splits into R1 and R2 (parallel)
# Voltage across R1 and R2 is the same: I_r1 * r1 = I_r2 * r2
prob_current += I_r1 * r1 == I_r2 * r2, "Voltage_R1_R2_Equal"
# Sum of currents through R1 and R2 equals the current flowing into this branch
prob_current += I_r1 + I_r2 == I_total, "KCL_R1_R2" # Assuming I_total is the current before the split

# Current through R3 is equal to the total current (series connection)
prob_current += I_r3 == I_total, "Current_R3_Total"

# Total current splits into R4 and R5 (parallel)
# Voltage across R4 and R5 is the same: I_r4 * r4 = I_r5 * r5
prob_current += I_r4 * r4 == I_r5 * r5, "Voltage_R4_R5_Equal"
# The sum of currents through R4 and R5 equals the current flowing into this branch
prob_current += I_r4 + I_r5 == I_total, "KCL_R4_R5" # Assuming I_total is the current before the split

# Note: The KCL_R1_R2 and KCL_R4_R5 constraints above assume that the entire I_total current
# flows into both parallel branches. In a real series connection, the current
# I_total first flows through R1||R2, then through R3, and then through R4||R5.
# Corrected KCL constraints:
# The input current I_total splits into R1 and R2. The sum of currents through R1 and R2 must be equal to I_total.
# The current through R3 is equal to I_total (series connection of R3 with the rest of the circuit).
# The current I_total splits into R4 and R5. The sum of currents through R4 and R5 must be equal to I_total.
# This means that the current I_total is the same in each series part.

# Corrected current constraints (according to series topology):
# Current through R3 is equal to the total current
prob_current += I_r3 == I_total, "Current_R3_is_Total"

# Total current (I_total) splits into R1 and R2. The sum of I_r1 and I_r2 must be equal to I_total.
prob_current += I_r1 + I_r2 == I_total, "KCL_R1_R2_sum"
# Voltage across R1 and R2 is the same.
prob_current += I_r1 * r1 == I_r2 * r2, "Voltage_R1_R2_equal"

# Total current (I_total) splits into R4 and R5. The sum of I_r4 and I_r5 must be equal to I_total.
prob_current += I_r4 + I_r5 == I_total, "KCL_R4_R5_sum"
# Voltage across R4 and R5 is the same.
prob_current += I_r4 * r4 == I_r5 * r5, "Voltage_R4_R5_equal"


# Solve the problem
print_solution(prob_current)

Status: Optimal
Current_R1 = 1.5428571
Current_R2 = 2.0571429
Current_R3 = 3.6
Current_R4 = 1.6
Current_R5 = 2.0
Total_Current = 3.6
Objective value = 3.6


# Challenge 2

In [9]:
import cvxpy as cp
import numpy as np

# Given nominal values
u1_nom, u2_nom, u3_nom, u4_nom, u5_nom = 6, 10, 4, 7, 3
# Nominal currents given in mA, convert to A
i1_nom_mA, i2_nom_mA, i3_nom_mA, i4_nom_mA, i5_nom_mA = 4, 2, 2, 2, 4

i1_nom = i1_nom_mA / 1000.0 # Conversion to A
i2_nom = i2_nom_mA / 1000.0
i3_nom = i3_nom_mA / 1000.0
i4_nom = i4_nom_mA / 1000.0
i5_nom = i5_nom_mA / 1000.0

# Current tolerance in A (1 mA)
delta_i = 1.0 / 1000.0 # 0.001 A


# Calculate nominal resistances (V/A = Ohm)
r1 = u1_nom / i1_nom if i1_nom != 0 else 1e9 # Using 1e9 for a very large resistance
r2 = u2_nom / i2_nom if i2_nom != 0 else 1e9
r3 = u3_nom / i3_nom if i3_nom != 0 else 1e9
r4 = u4_nom / i4_nom if i4_nom != 0 else 1e9
r5 = u5_nom / i5_nom if i5_nom != 0 else 1e9

print(f"Nominal resistances: R1={r1} Ohm, R2={r2} Ohm, R3={r3} Ohm, R4={r4} Ohm, R5={r5} Ohm")

# Decision variables (currents through resistors in A)
i1 = cp.Variable()
i2 = cp.Variable()
i3 = cp.Variable()
i4 = cp.Variable()
i5 = cp.Variable()

# Variable for total current (optional, can be eliminated by KCL)
i_total = cp.Variable()

# Objective function: minimize sum of dissipated power (I^2 * R)
# In CVXPY, the square of a variable is a valid form for a quadratic objective function.
objective = cp.Minimize(r1 * i1**2 + r2 * i2**2 + r3 * i3**2 + r4 * i4**2 + r5 * i5**2)

# Constraints:
constraints = [
    # Current variation constraints (nominal +/- delta_i in A)
    i1_nom - delta_i <= i1, i1 <= i1_nom + delta_i,
    i2_nom - delta_i <= i2, i2 <= i2_nom + delta_i,
    i3_nom - delta_i <= i3, i3 <= i3_nom + delta_i,
    i4_nom - delta_i <= i4, i4 <= i4_nom + delta_i,
    i5_nom - delta_i <= i5, i5 <= i5_nom + delta_i,

    # Kirchhoff's Current Laws (KCL) - based on the bridge description
    i1 + i2 == i_total,       # KCL at the input node
    i1 == i3 + i4,            # KCL after R1
    i2 + i3 == i5,            # KCL before R5
    i4 + i5 == i_total,       # KCL at the output node (return)
    # Note that from KCL, i_total = i1 + i2 and i_total = i4 + i5.
    # We can remove i_total as a variable and substitute it into the constraints.
    # constraints.append(i1 + i2 == i4 + i5) # Alternative formulation without i_total
]

# Create the CVXPY problem
problem = cp.Problem(objective, constraints)

# Solve the problem using the OSQP solver (should work by default for QP)
# If OSQP does not work, try 'ECOS' or 'CVXOPT'
try:
    problem.solve(solver=cp.OSQP)
except cp.SolverError:
    print("OSQP solver did not find a solution. Trying ECOS.")
    try:
        problem.solve(solver=cp.ECOS)
    except cp.SolverError:
        print("ECOS solver also did not find a solution. Try another solver.")
        print(problem.status)


# Check solution status and display results
print("\nSolution status:", problem.status)

if problem.status in ["optimal", "optimal_near"]:
    print("Minimum dissipated power (W):", problem.value) # Changed unit in text
    print("Optimal current values (mA):")
    print("i1 =", i1.value * 1000.0) # Convert to mA
    print("i2 =", i2.value * 1000.0) # Convert to mA
    print("i3 =", i3.value * 1000.0) # Convert to mA
    print("i4 =", i4.value * 1000.0) # Convert to mA
    print("i5 =", i5.value * 1000.0) # Convert to mA
    print("Total current (according to KCL) (mA):", (i_total.value if i_total.value is not None else i1.value + i2.value) * 1000.0) # Convert to mA

elif problem.status == "infeasible":
    print("Problem is infeasible. No currents within the given ranges satisfy KCL.")
elif problem.status == "unbounded":
    print("Problem is unbounded (which should not happen in this physical problem with a correct formulation).")
else:
    print("Could not find an optimal solution.")
    print("Solver status:", problem.status)

Nominal resistances: R1=1500.0 Ohm, R2=5000.0 Ohm, R3=2000.0 Ohm, R4=3500.0 Ohm, R5=750.0 Ohm

Solution status: optimal
Minimum dissipated power (W): 0.03675000000000002
Optimal current values (mA):
i1 = 3.000000000000001
i2 = 0.9999999999999998
i3 = 2.000000000000001
i4 = 1.0000000000000002
i5 = 3.0000000000000004
Total current (according to KCL) (mA): 4.000000000000001
